In [1]:
import pandas as pd # 데이터프레임 사용을 위해
from math import log # IDF 계산을 위해

In [2]:
df= pd.DataFrame(pd.read_csv('coron.csv', encoding='cp949'))
df.head()

,Unnamed: 0,years,company,title,contents,link
0,1,2020.09.09.,OhmyNews,"코로나19 시대, 친환경 '온라인 녹색매장' 열린다",제공대형마트 등에서 운영하던 '녹색매장'이 온라인 인터파크(www.interpark...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,2,2020.09.09.,헤럴드경제,"“친환경 제품, 온라인 녹색매장서 구매하세요”",환경산업기술원 녹색제품 기획전환경부 산하 한국환경산업기술원(원장 유제철·사진)은 9...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,3,2020.09.10.,ZDNet Korea,"쿠팡, 소상공인 온라인 판로 지원","농림축산식품부, 한국농수산식품유통공사와 농공상기업 돕는 기획전 진행. 쿠팡이 코로나...",https://news.naver.com/main/read.nhn?mode=LSD&...
3,4,2020.09.09.,financial news,"서울시, 소상공인·전통시장 온라인 진출 지원한다",서울시는 온라인쇼핑 운영 경험이 많지 않은 소상공인과 전통시장 상인을 대상으로 온라...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,5,2020.09.09.,뉴스1,"서울시, '언택트 시대' 소상공인·전통시장 상인 온라인 진출 지원",소상공인 300곳 쿠팡 등 입점 지원…판매수수료 2~5%p 인하전통시장 온라인시장 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [3]:
import re 
# '한글'을 제외한 다른 문자 모두 제거
remove_except_ko = re.compile(r"[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]")
def preprocess(text):
    text = re.sub(remove_except_ko,' ',text).strip()
    return text

df['title'] = df['title'].map(lambda x : preprocess(x))
df['contents'] = df['contents'].map(lambda x : preprocess(x))
df.head()

,Unnamed: 0,years,company,title,contents,link
0,1,2020.09.09.,OhmyNews,코로나 시대 친환경 온라인 녹색매장 열린다,제공대형마트 등에서 운영하던 녹색매장 이 온라인 인터파크 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,2,2020.09.09.,헤럴드경제,친환경 제품 온라인 녹색매장서 구매하세요,환경산업기술원 녹색제품 기획전환경부 산하 한국환경산업기술원 원장 유제철 사진 은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,3,2020.09.10.,ZDNet Korea,쿠팡 소상공인 온라인 판로 지원,농림축산식품부 한국농수산식품유통공사와 농공상기업 돕는 기획전 진행 쿠팡이 코로나...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,4,2020.09.09.,financial news,서울시 소상공인 전통시장 온라인 진출 지원한다,서울시는 온라인쇼핑 운영 경험이 많지 않은 소상공인과 전통시장 상인을 대상으로 온라...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,5,2020.09.09.,뉴스1,서울시 언택트 시대 소상공인 전통시장 상인 온라인 진출 지원,소상공인 곳 쿠팡 등 입점 지원 판매수수료 인하전통시장 온라인시장 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [4]:
# Mecab
from konlpy.tag import Mecab
# 선언
mecab= Mecab()

In [5]:
stop_words = ['은','는','이','가','을','를','께서','에서','이다','에게','으로','이랑','까지','부터','하다',
             '기자','뉴스','무단','전재','및','재배포','금지','저작권','사진','김병기','앵커','이연춘','연합뉴스',
             '파이낸셜뉴스','습니다','입니다']
stopPos = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JC','JX',
           'SF','SE','SS','SP','SSO','SSC','SC','SY','SH','SL','SN',
           'MM','MAG','MAJ']
def token_mecab(text):
    text = mecab.pos(text)
    text = [i for i in text if len(i[0])>1]
    text = [i for i in text if i[0] not in stop_words]
    text = [i[0] for i in text if i[1] not in stopPos]
    return text

In [6]:
def make_tokens(df):
    df['tokens'] = ''
    tokens_list=[]
    for i in range(len(df)):
        token = token_mecab(df['contents'][i])
        df['tokens'][i] = ' '.join(token)
    return df

df = make_tokens(df)

C:\Data\envs\py37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df.head()

,Unnamed: 0,years,company,title,contents,link,tokens
0,1,2020.09.09.,OhmyNews,코로나 시대 친환경 온라인 녹색매장 열린다,제공대형마트 등에서 운영하던 녹색매장 이 온라인 인터파크 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,공대 마트 운영 녹색 온라인 인터 파크 열렸 올해 코로나 인해 온라인 쇼핑몰 이용자...
1,2,2020.09.09.,헤럴드경제,친환경 제품 온라인 녹색매장서 구매하세요,환경산업기술원 녹색제품 기획전환경부 산하 한국환경산업기술원 원장 유제철 사진 은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,환경 산업 기술원 녹색 제품 기획 전환 경부 산하 한국 환경 산업 기술원 원장 유제...
2,3,2020.09.10.,ZDNet Korea,쿠팡 소상공인 온라인 판로 지원,농림축산식품부 한국농수산식품유통공사와 농공상기업 돕는 기획전 진행 쿠팡이 코로나...,https://news.naver.com/main/read.nhn?mode=LSD&...,농림 축산 식품 한국 농수산 식품 유통 공사 농공 기업 기획 진행 쿠팡 코로나 판로...
3,4,2020.09.09.,financial news,서울시 소상공인 전통시장 온라인 진출 지원한다,서울시는 온라인쇼핑 운영 경험이 많지 않은 소상공인과 전통시장 상인을 대상으로 온라...,https://news.naver.com/main/read.nhn?mode=LSD&...,서울시 온라인 쇼핑 운영 경험 소상 공인 전통 시장 상인 대상 온라인 시장 진출 지...
4,5,2020.09.09.,뉴스1,서울시 언택트 시대 소상공인 전통시장 상인 온라인 진출 지원,소상공인 곳 쿠팡 등 입점 지원 판매수수료 인하전통시장 온라인시장 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,상공 쿠팡 입점 지원 판매 수수료 인하 전통 시장 온라인 시장 진입 는다 온라인 장...


In [27]:
# 공백으로 토큰 분리
def my_tokenizer(text):
    return text.split()

In [28]:
docs=list(df['tokens'])
docs

['공대 마트 운영 녹색 온라인 인터 파크 열렸 올해 코로나 인해 온라인 쇼핑몰 이용자 급증 따른 환경부 장관 조명래 한국 환경 산업 기술원 원장 유제철 온라인 녹색 제품 환경 표지 인증 제품 확인 구매 도록 온라인 녹색 매장 시범 운영 한다 환경 표지 인증 동일 용도 제품 제품 과정 단계 걸쳐 에너지 자원 소비 줄이 오염 물질 발생 최소 제품 환경 표지 인증 국가 공인 제도 환경부 녹색 소비 활성 위해 녹색 제품 대한 정보 안내 구매 편의 위해 노력 매장 지정 녹색 매장 운영 지난 현재 백화점 대형 마트 편의점 매장 운영 이번 온라인 녹색 매장 녹색 제품 전용 홍보 운영 제품 종류 분류 녹색 제품 구매 온라인 녹색 매장 운영 기념 제품 구매 최대 할인 지급 한다 환경부 녹색 제품 전용 홍보 일상 생활 구매 사무 기기 주방 용품 비롯 생활 용품 가전 기기 환경 표지 인증 다양 제품 전시 면서 구매 화면 상단 녹색 제품 정보 시스템 접속 단추 소비자 해당 제품 환경 정보 확인 밝혔 환경부 한국 환경 산업 기술원 이번 온라인 녹색 매장 운영 통해 쇼핑몰 이용 고객 구매 성향 분석 맞춤 녹색 제품 제공 방안 마련 한국 온라인 쇼핑 협회 온라인 쇼핑몰 업체 온라인 녹색 매장 지속 확대 한다 김동구 환경부 녹색 전환 정책 녹색 제품 제품 생산 폐기 이르 과정 환경 고려 친환경 제품 라면서 늘어나 온라인 쇼핑 수요 반영 소비자 다양 녹색 제품 접할 도록 온라인 판매 창구 늘리',
 '환경 산업 기술원 녹색 제품 기획 전환 경부 산하 한국 환경 산업 기술원 원장 유제철 온라인 녹색 제품 확인 구매 도록 인터 파크 이날 온라인 녹색 매장 시범 운영 한다고 밝혔 이번 시범 운영 올해 코로나 영향 온라인 쇼핑몰 이용 소비자 급증 따라 그동안 대형 마트 운영 녹색 매장 온라인 확대 녹색 매장 지정 제도 녹색 소비 활성 위해 녹색 제품 대한 정보 안내 구매 편의 위해 노력 매장 환경부 지정 제도 백화점 대형 마트 편의점 현재 매장이 지정 녹색 매장 녹색 제품 전용 홍보 운영 제품 

In [29]:
doc_ls =[doc.split() for doc in docs]
doc_ls

[['공대',
  '마트',
  '운영',
  '녹색',
  '온라인',
  '인터',
  '파크',
  '열렸',
  '올해',
  '코로나',
  '인해',
  '온라인',
  '쇼핑몰',
  '이용자',
  '급증',
  '따른',
  '환경부',
  '장관',
  '조명래',
  '한국',
  '환경',
  '산업',
  '기술원',
  '원장',
  '유제철',
  '온라인',
  '녹색',
  '제품',
  '환경',
  '표지',
  '인증',
  '제품',
  '확인',
  '구매',
  '도록',
  '온라인',
  '녹색',
  '매장',
  '시범',
  '운영',
  '한다',
  '환경',
  '표지',
  '인증',
  '동일',
  '용도',
  '제품',
  '제품',
  '과정',
  '단계',
  '걸쳐',
  '에너지',
  '자원',
  '소비',
  '줄이',
  '오염',
  '물질',
  '발생',
  '최소',
  '제품',
  '환경',
  '표지',
  '인증',
  '국가',
  '공인',
  '제도',
  '환경부',
  '녹색',
  '소비',
  '활성',
  '위해',
  '녹색',
  '제품',
  '대한',
  '정보',
  '안내',
  '구매',
  '편의',
  '위해',
  '노력',
  '매장',
  '지정',
  '녹색',
  '매장',
  '운영',
  '지난',
  '현재',
  '백화점',
  '대형',
  '마트',
  '편의점',
  '매장',
  '운영',
  '이번',
  '온라인',
  '녹색',
  '매장',
  '녹색',
  '제품',
  '전용',
  '홍보',
  '운영',
  '제품',
  '종류',
  '분류',
  '녹색',
  '제품',
  '구매',
  '온라인',
  '녹색',
  '매장',
  '운영',
  '기념',
  '제품',
  '구매',
  '최대',
  '할인',
  '지급',
  '한다',
  '환경부',
  '녹색',
  '제품',
  '전용'

In [30]:
from collections import defaultdict

wordid= defaultdict(lambda : len(wordid))
wordid

defaultdict(<function __main__.<lambda>()>, {})

In [31]:
for doc in doc_ls:
    for token in doc:
        wordid[token]
wordid

defaultdict(<function __main__.<lambda>()>,
            {'공대': 0,
             '마트': 1,
             '운영': 2,
             '녹색': 3,
             '온라인': 4,
             '인터': 5,
             '파크': 6,
             '열렸': 7,
             '올해': 8,
             '코로나': 9,
             '인해': 10,
             '쇼핑몰': 11,
             '이용자': 12,
             '급증': 13,
             '따른': 14,
             '환경부': 15,
             '장관': 16,
             '조명래': 17,
             '한국': 18,
             '환경': 19,
             '산업': 20,
             '기술원': 21,
             '원장': 22,
             '유제철': 23,
             '제품': 24,
             '표지': 25,
             '인증': 26,
             '확인': 27,
             '구매': 28,
             '도록': 29,
             '매장': 30,
             '시범': 31,
             '한다': 32,
             '동일': 33,
             '용도': 34,
             '과정': 35,
             '단계': 36,
             '걸쳐': 37,
             '에너지': 38,
             '자원': 39,
             '소비': 40,
             '

In [32]:
import numpy as np

In [33]:
DTM = np.zeros((len(doc_ls), len(wordid)), dtype=int)
print(DTM)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [34]:
for i, doc in enumerate(doc_ls):
    for token in doc:
        DTM[i, wordid[token]] += 1 # 해당 토큰의 위치 (columns)
# 행렬로 표기 (BOW와 차이점: BOW는 1차원적 배열)
DTM

array([[1, 2, 7, ..., 0, 0, 0],
       [0, 2, 6, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 1]])

In [35]:
def computeTF(DTM):
    doc_len = len(DTM) # 문서개수 2개
    word_len = len(DTM[0]) # 토큰의 개수 8개
    # tf를 계산하기 전 0으로 셋팅
    tf = np.zeros((doc_len, word_len))
    print(tf)
    # TF 계산 특정단어등장빈도/문서 내 전체등장단어빈도
    for doc_i in range(doc_len) :
        for word_i in range(word_len) :
            tf[doc_i, word_i] = DTM[doc_i, word_i]/DTM[doc_i].sum()
    return tf

In [36]:
computeTF(DTM)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


array([[0.00440529, 0.00881057, 0.030837  , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.01098901, 0.03296703, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00469484, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00847458, 0.00847458,
        0.00847458]])

In [37]:
import math
# IDF 계산 : log (총문서수/단어가등장한문서수)
def computeIDF(DTM):
    doc_len = len(DTM)
    word_len = len(DTM[0])
    
    idf = np.zeros(word_len)
    
    for i in range(word_len) :
        idf[i] = math.log10(doc_len/ np.count_nonzero(DTM[:,i]))
    return idf

In [38]:
computeIDF(DTM)

array([1.38021124, 0.68124124, 0.30103   , ..., 1.38021124, 1.38021124,
       1.38021124])

In [39]:
# TF-IDF 곱
def computeTFIDF(DTM):
    tf = computeTF(DTM)
    idf = computeIDF(DTM)
    tfidf = np.zeros(tf.shape)
    for doc_i in range(tf.shape[0]) :
        for word_i in range(tf.shape[1]) :
            tfidf[doc_i, word_i] = tf[doc_i, word_i] * idf[word_i]
    return tfidf

In [40]:
computeTFIDF(DTM)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


array([[0.00608023, 0.00600213, 0.00928286, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00748617, 0.00992407, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00141329, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.01169671, 0.01169671,
        0.01169671]])

In [41]:
import pandas as pd

sorted_vocab = sorted((value,key)for key, value in wordid.items())
print(sorted_vocab)

[(0, '공대'), (1, '마트'), (2, '운영'), (3, '녹색'), (4, '온라인'), (5, '인터'), (6, '파크'), (7, '열렸'), (8, '올해'), (9, '코로나'), (10, '인해'), (11, '쇼핑몰'), (12, '이용자'), (13, '급증'), (14, '따른'), (15, '환경부'), (16, '장관'), (17, '조명래'), (18, '한국'), (19, '환경'), (20, '산업'), (21, '기술원'), (22, '원장'), (23, '유제철'), (24, '제품'), (25, '표지'), (26, '인증'), (27, '확인'), (28, '구매'), (29, '도록'), (30, '매장'), (31, '시범'), (32, '한다'), (33, '동일'), (34, '용도'), (35, '과정'), (36, '단계'), (37, '걸쳐'), (38, '에너지'), (39, '자원'), (40, '소비'), (41, '줄이'), (42, '오염'), (43, '물질'), (44, '발생'), (45, '최소'), (46, '국가'), (47, '공인'), (48, '제도'), (49, '활성'), (50, '위해'), (51, '대한'), (52, '정보'), (53, '안내'), (54, '편의'), (55, '노력'), (56, '지정'), (57, '지난'), (58, '현재'), (59, '백화점'), (60, '대형'), (61, '편의점'), (62, '이번'), (63, '전용'), (64, '홍보'), (65, '종류'), (66, '분류'), (67, '기념'), (68, '최대'), (69, '할인'), (70, '지급'), (71, '일상'), (72, '생활'), (73, '사무'), (74, '기기'), (75, '주방'), (76, '용품'), (77, '비롯'), (78, '가전'), (79, '다양'), (80, '전시'), (81, '면서'), (82, '화면'), (8

In [42]:
vocab = [v[1] for v in sorted_vocab]
print(vocab)

['공대', '마트', '운영', '녹색', '온라인', '인터', '파크', '열렸', '올해', '코로나', '인해', '쇼핑몰', '이용자', '급증', '따른', '환경부', '장관', '조명래', '한국', '환경', '산업', '기술원', '원장', '유제철', '제품', '표지', '인증', '확인', '구매', '도록', '매장', '시범', '한다', '동일', '용도', '과정', '단계', '걸쳐', '에너지', '자원', '소비', '줄이', '오염', '물질', '발생', '최소', '국가', '공인', '제도', '활성', '위해', '대한', '정보', '안내', '편의', '노력', '지정', '지난', '현재', '백화점', '대형', '편의점', '이번', '전용', '홍보', '종류', '분류', '기념', '최대', '할인', '지급', '일상', '생활', '사무', '기기', '주방', '용품', '비롯', '가전', '다양', '전시', '면서', '화면', '상단', '시스템', '접속', '단추', '소비자', '해당', '밝혔', '통해', '이용', '고객', '성향', '분석', '맞춤', '제공', '방안', '마련', '쇼핑', '협회', '업체', '지속', '확대', '김동구', '전환', '정책', '생산', '폐기', '이르', '고려', '친환경', '라면서', '늘어나', '수요', '반영', '접할', '판매', '창구', '늘리', '기획', '경부', '산하', '이날', '한다고', '영향', '따라', '그동안', '매장이', '계획', '농림', '축산', '식품', '농수산', '유통', '공사', '농공', '기업', '진행', '쿠팡', '판로', '어려움', '소상', '지역', '농가', '나섰', '중소', '상공', '희망', '재단', '서울시', '손잡', '익숙', '전통', '시장', '상인', '진출', '신장', '지원', '입점', '원하', '방법', '기회'

In [43]:
tfidf = computeTFIDF(DTM)
pd.DataFrame(tfidf, columns=vocab)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


,공대,마트,운영,녹색,온라인,인터,파크,열렸,올해,코로나,...,자동차,으나,직격탄,연속,원대,보였,통신,는데,커지,상승
0,0.00608,0.006002,0.009283,0.085574,0.0,0.003001,0.003001,0.00608,0.001675,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.00000,0.007486,0.009924,0.094873,0.0,0.003743,0.003743,0.00000,0.002089,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.00000,0.000000,0.001413,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.00000,0.000000,0.007342,0.000000,0.0,0.003323,0.003323,0.00000,0.001855,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.00000,0.000000,0.006424,0.000000,0.0,0.002077,0.002077,0.00000,0.002318,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.00000,0.000000,0.002618,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.001901,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.001720,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.00000,0.000000,0.002700,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.00000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.001184,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [55]:
TFIDF = pd.DataFrame(tfidf, columns=vocab)
TFIDF.head()

,공대,마트,운영,녹색,온라인,인터,파크,열렸,올해,코로나,...,자동차,으나,직격탄,연속,원대,보였,통신,는데,커지,상승
0,0.00608,0.006002,0.009283,0.085574,0.0,0.003001,0.003001,0.00608,0.001675,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.007486,0.009924,0.094873,0.0,0.003743,0.003743,0.00000,0.002089,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.000000,0.001413,0.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.000000,0.007342,0.000000,0.0,0.003323,0.003323,0.00000,0.001855,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.000000,0.006424,0.000000,0.0,0.002077,0.002077,0.00000,0.002318,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
TFIDF.max(axis=0).max()

0.09487307657561538

In [57]:
# csv 파일로 저장
#TFIDF.to_csv('C:/Users/jminw/Desktop/tf-idf.csv')